# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [73]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

## Load the Data

Start by importing the data from the pickle file.

In [74]:
train_path = '/Users/blakejacquot/Dropbox/MOOCs/Udacity_SelfDrivingCar/Term1/Keras-Data/train.p'
test_path = '/Users/blakejacquot/Dropbox/MOOCs/Udacity_SelfDrivingCar/Term1/Keras-Data/test.p'

with open(train_path, 'rb') as f:
    train_data = pickle.load(f)

with open(test_path, 'rb') as f:
    test_data = pickle.load(f)
    
print(type(train_data))
print(train_data.keys())
    
X_train, y_train = train_data['features'], train_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

print(type(X_train), type(y_train))
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

<class 'dict'>
dict_keys(['labels', 'features'])
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(39209, 32, 32, 3) (39209,)
(12630, 32, 32, 3) (12630,)


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [75]:
# TODO: Use `train_test_split` here.
print('Train shape before validation split', X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=0.05, random_state=101)
print('Train shape after validation split', X_train.shape, y_train.shape)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

Train shape before validation split (39209, 32, 32, 3) (39209,)
Train shape after validation split (37248, 32, 32, 3) (37248,)


## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5 and has a mean of 0.

In [76]:
# TODO: Implement data normalization here.

def normalize_set(x):
    print('Normalizing data')
    x_shape = x.shape
    num_el = x_shape[0]
    ret_images = np.ones((x_shape[0],x_shape[1],x_shape[2], x_shape[3]))
    for i in range(num_el):
        curr_im = x[i,:,:,:]
        empty_im = ret_images[i, :, :, :]
        curr_im = curr_im / 255.0
        curr_im = curr_im * 1.0
        curr_im = curr_im - 0.5
        proc_im = curr_im
        ret_images[i, :, :, :] = proc_im
    return ret_images

print(np.min(X_train), np.max(X_train))
print(X_train.shape)

X_train = normalize_set(X_train)
X_val = normalize_set(X_val)
X_test = normalize_set(X_test)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the training data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(round(np.mean(X_val)) == 0), "The mean of the validation data is: %f" % np.mean(X_val)
assert(np.min(X_val) == -0.5 and np.max(X_val) == 0.5), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

0 255
(37248, 32, 32, 3)
Normalizing data
Normalizing data
Normalizing data


## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [77]:
# TODO: Build a two-layer feedforward neural network with Keras here.
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

X_shape = X_train.shape
print(X_shape[1],X_shape[2], X_shape[3])

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils


model = Sequential()
#model.add(Dense(128, input_shape=(3072,)))
model.add(Dense(128, input_shape=(3072,)))
model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Dense(43))
model.add(Activation('relu'))
#model.add(Dropout(0.2))
#model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

32 32 3
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_9 (Dense)                  (None, 128)           393344      dense_input_5[0][0]              
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 128)           0           dense_9[0][0]                    
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 43)            5547        activation_13[0][0]              
____________________________________________________________________________________________________
activation_14 (Activation)       (None, 43)            0           dense_10[0][0]                   
___________________________________________________________________________________

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [80]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

XX_train = X_train
XX_test = X_test
XX_val = X_val

# TODO: Compile and train the model here.
batch_size = 128
nb_classes = 43
nb_epoch = 2

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Reshape train data
X_shape = XX_train.shape
print('X train shape = ', X_shape)
z = X_shape[1] * X_shape[2] * X_shape[3]
XX_train = XX_train.reshape(X_shape[0],z)
XX_train = XX_train.astype('float32')
print('X_train shape = ', XX_train.shape)

# Reshape test data
X_shape = XX_test.shape
print('X_test shape = ', X_shape)
z = X_shape[1] * X_shape[2] * X_shape[3]
XX_test = XX_test.reshape(X_shape[0],z)
XX_test = XX_test.astype('float32')
print('X_test shape = ', XX_test.shape)

# Reshape val data
X_shape = XX_val.shape
print('X_val shape = ', X_shape)
z = X_shape[1] * X_shape[2] * X_shape[3]
XX_val = XX_val.reshape(X_shape[0],z)
XX_val = XX_val.astype('float32')
print('X_val shape = ', XX_val.shape)

print('Training model')
history = model.fit(XX_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(XX_val, Y_val))

print('Evaluating model')
score = model.evaluate(XX_test, Y_test, verbose=0)

print('Test score:', score[0])
print('Test accuracy:', score[1])

print('Training accuracy = ', history.history['acc'][-1])
print('Validation accuracy = ', history.history['val_acc'][-1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

X train shape =  (37248, 32, 32, 3)
X_train shape =  (37248, 3072)
X_test shape =  (12630, 32, 32, 3)
X_test shape =  (12630, 3072)
X_val shape =  (1961, 32, 32, 3)
X_val shape =  (1961, 3072)
Training model
Train on 37248 samples, validate on 1961 samples
Epoch 1/2
37248/37248 [==============================] - 5s - loss: 0.6537 - acc: 0.8378 - val_loss: 0.6323 - val_acc: 0.8445
Epoch 2/2
37248/37248 [==============================] - 5s - loss: 0.5888 - acc: 0.8552 - val_loss: 0.6047 - val_acc: 0.8358
Evaluating model
Test score: 0.982883561422
Test accuracy: 0.748693586727
Training accuracy =  0.855213702749
Validation accuracy =  0.835798062365


AssertionError: The training accuracy was: 0.855

**Validation Accuracy**: (fill in here)

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [ ]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.

# TODO: Compile and train the model here.

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [ ]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.

# TODO: Compile and train the model here.

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [ ]:
# TODO: Re-construct the network and add dropout after the pooling layer.

# TODO: Compile and train the model here.

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: (fill in here)

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
with open('./test.p', mode='rb') as f:
    test = pickle.load(f)
    
X_test = test['features']
y_test = test['labels']
X_test = X_test.astype('float32')
X_test /= 255
X_test -= 0.5
Y_test = np_utils.to_categorical(y_test, 43)

model.evaluate(X_test, Y_test)

**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.